In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import sqlite3
from datetime import datetime 

# Initialise variables

url = 'https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks'
table_attribs = ['Name', 'MC_USD_Billion']
db_name = 'Banks.db'
table_name = 'Largest_banks'
csv_path = './Largest_banks_data.csv'
sql_connection = sqlite3.connect('Largest_banks.db')

In [9]:
def extract(url, table_attribs):
    # Extracts the required information from the website and saves it to a dataframe

    page = requests.get(url).text
    data = BeautifulSoup(page, 'html.parser')
    df = pd.DataFrame(columns=table_attribs)
    tables = data.find_all('tbody')
    rows = tables[0].find_all('tr')
    print(rows)


In [ ]:
'''
    for row in rows:
        col = row.find_all('td')
        if len(col)!=0: # if the column has information:
            # Here we check if the 2 columns that we are extracting have information, if not,the row will be ignored
            data_dict = {"Name": col[1].a.contents[0],
                            "MC_USD_Billion": col[2].contents[0]}
            df1 = pd.DataFrame(data_dict, index=[0])
            df = pd.concat([df,df1], ignore_index=True)
    return df
'''